# Introduction to CAOM

To enable future growth and enhance the ability of users to find data of interest to their science case, the CADC has developed the **Common Archive Observation Model (CAOM)**. CAOM (now at version 2.2) enables the storage of observational metadata from the complete set of telescopic data and searching through that metadata using a single interface. The generalized capability of CAOM comes at the expense of some model complexity and the requirement of adopting a language that is unfamiliar to users. To decrease the learning curve for users, the CADC exposes CAOM via a simplified search web page interface (AdvancedSearch), this same interface provides access to all observations stored in the archive. For users requiring access to more details of the observations and greater flexibility in query construction CAOM is also exposed via a Table Access Protocol (TAP) web service that can be accessed via tools like ‘Topcat’ or directly via any http aware client. CAOM greatly decreases the workload involved in maintaining an archive but still requires expert knowledge to ensure that the metadata is correctly translated from the observational record into the CAOM database system.

CAOM metadata records that describe a complete observation can be constructed using two pieces of software. Each can be used independently and users are not required to use both. Before beginning with creating a record, however, one should understand the structure of the metadata model.

CAOM consists of a database structure that describe the circumstances (Timing, Location and Wavelength/Frequency) of an observation, the nature observation (image, spectrograph, polarizing, receiver, etc.) and the processing level of different files associated with the observation (ie. the raw and calibrated data are stored in the same CAOM entry).

The top level of the model is the Observation. The Observation is the overall container for all associated datasets. Each associated data set is stored in a Plane, so an Observation is made up of multiple Planes data. Within each Plane are the actual data files that contain the observational data, the Artifacts of the Plane, a FITS file is a good example of an Artifact. Any complex data structures within an Artifact, such a Multi Extension FITS file, is further described by having each structure within the Artifact described as a Part. A single Artifact can contain multiple Parts. A Part within an Artifact is normally describable and findable via metadata stored within the Artifact (such as the headers of an MEF file). Sometimes, very very rarely, a Part might itself contain complex data structures that are not clearly separated by metadata within the file, each of the data structures (normally there is only one) within a Part is called a Chunk. Thus we have:

in
Observation

  ```-> Plane 
    -> Artifact
        -> Part 
            -> Chunk  
        -> Part 
            -> Chunk 
        -> Part 
            ...
  -> Plane
    -> Artifact 
        ... 
  -> Plane 
     ...```
     
A technical description of CAOM, including a UML, is available from [CAOM Reference Documentation](http://www.opencadc.org/caom2/)

To create a CAOM record for transfer into the CADC database search system can be achieved using the software too fits2caom2 or by developing your own software in Python base on the pyCAOM2 module or both! The best place to start is likely to examine an existing set of CAOM2 records as examples, likely ones that are for observations that are similar to those you are attempting to store within the CADC.




# Tutorial
This tutorial describes the mechanism for accessing ALMA metadata and creating corresponding CAOM2 metadata. The Python code below is dependent on the following packages all of which available in pypi and installable with the command `pip install <package>`:
1. `astropy` - [astropy](https://github.com/astropy/astropy)
2. `astroquery` - [astroquery](https://github.com/astropy/astroquery)
3. `caom2` - [caom2](https://github.com/opencadc/caom2tools)

## Step 1 - Model the data
One possible model is to map every Member ous id to a plane. **TBD: reasoning and possible discussion**

In [6]:
from astroquery.alma import Alma
results = Alma.query({'member_ous_id': 'uid://A001/X11a2/X11'}, science=False)
results.show_in_notebook()